## Домашнее задание №2
### Выполнили: Алексеев Антон, Булатов Артур, Корнеева Елизавета ИАД1

#### Краткая постановка задачи и формулировка задания:

Домашнее задание заключается в предсказании роста цен акции компании Газпром по новостям о компании.
Для этого, первым делом, нужно предобработать входные данные, затем построить модель, затем нужно попробовать усовершенствовать свои предсказания любыми законными способами

#### Описание используемых инструментов:

pandas — программная библиотека на языке Python для обработки и анализа данных. Pandas предоставляет специальные структуры данных и операции для манипулирования числовыми таблицами и временными рядами.
pymorphy2 - морфологический анализатор для русского языка, написанный на языке Python и использующий словари из OpenCorpora. Он работает под Python 2.6, 2.7, 3.2 и 3.3 и распространяется по лицензии MIT.

#### Краткий план работы программы:

   - Часть первая "Вводная"
        - Предобработка текста – убираем пунктуацию, стоп-слова и приводим к начальной форме.
   - Часть вторая "Классификационная"
        - Baseline tf-idf + MultinomialNB
        - Подбор гиперпараметров GridSearchCV
   - Часть третья "Творческая "
        - Пробуем оставить только существительные
        - Пробуем оставить только существительные + глаголы
        - Пробуем оставить только слова из топиков LDA

Более подробно каждый шаг расписан в виде комментариев к коду

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from string import punctuation
import re
import pymorphy2
from numpy import corrcoef
from datetime import datetime
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder  
from sklearn import metrics
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as lda

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

In [2]:
texts = pd.read_csv('texts.csv')
texts.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [3]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all.head(5)

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,08.12.2017,"133,43000","132,60000","133,90000","132,00000",16037970
1,07.12.2017,"133,70000","133,02000","133,87000","132,81000",18198430
2,06.12.2017,"133,33000","134,00000","134,29000","132,91000",14641730
3,05.12.2017,"133,48000","133,65000","133,99000","132,78000",12684800
4,04.12.2017,"133,01000","133,77000","134,00000","131,93000",17818980


In [4]:
texts.rename(columns={'text': 'text_original'}, inplace=True)
texts['text'] = texts['text_original']

# предобработка текста
for i in tqdm(range(len(texts['text_original'][:]))):
    # убираем стоп-слова
    s = ' '.join([word for word in texts['text_original'][i].split() if word not in stopwords.words('russian')])
    # убираем пунктуацию
    s = ''.join(c for c in s if c not in punctuation)

    # приводим слова в начальную форму
    pymorph = pymorphy2.MorphAnalyzer()
    words = []
    for word in s.split():
        word = pymorph.parse(word)[0].normal_form
        words.append(word)
    texts['text'][i] = ' '.join(words)

100%|██████████| 1203/1203 [04:27<00:00,  4.51it/s]


In [5]:
texts.head()

,date,text_original,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...,компания рассчитывать решение газовый спор укр...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова...",как предполагать “ъ” «газпром» воспользоваться...
2,01.11.2017,Новая редакция американских санкций ставит по...,новый редакция американский санкция ставить уг...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри...",как стать известно “ъ” известный рынок рискмен...
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за...",новатэк который пять год собираться запустить ...


In [6]:
# переводим цену закрытия из строки во float для работы с числами
pr_all['ClosingPrice'] = [float(i.replace(',', '.')) for i in pr_all['ClosingPrice'][:]]
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,08.12.2017,"133,43000",132.60,"133,90000","132,00000",16037970
1,07.12.2017,"133,70000",133.02,"133,87000","132,81000",18198430
2,06.12.2017,"133,33000",134.00,"134,29000","132,91000",14641730
3,05.12.2017,"133,48000",133.65,"133,99000","132,78000",12684800
4,04.12.2017,"133,01000",133.77,"134,00000","131,93000",17818980


In [7]:
# соединяем данные по дате
texts.rename(columns={'date': 'Date'}, inplace=True)
result = pd.merge(texts, pr_all, on='Date')

### Есть ли корреляция между средней длинной текста за день и ценой закрытия?

In [8]:
from numpy import corrcoef
coef = corrcoef(result['ClosingPrice'][:], [len(t) for t in result['text'][:]])[0, 1]
print("Коеффициент корреляции Пирсона между ценой закрытия и длинной текста – ", coef)

Коеффициент корреляции Пирсона между ценой закрытия и длинной текста –  0.01370484309179625


Судя по коэффициенту корреляции - цена закрытия не зависит от средней длины текста за день

### Есть ли корреляция между количеством упоминаний Алексея Миллера и ценой закрытия?

In [9]:
pattern_miller = re.compile(r'(Миллер)', re.IGNORECASE)
res = []
for i in range(len(result['text'][:])):
    search_str = texts['text'][i]
    res = re.findall(pattern_miller, search_str)
    res.append(len(res))

    
coef = corrcoef(result['ClosingPrice'][:], [len(re.findall(pattern_miller, t)) for t in result['text'][:]])[0, 1]
print("Коеффициент корреляции Пирсона между ценой закрытия и упоминаниями Миллера – ", coef)

Коеффициент корреляции Пирсона между ценой закрытия и упоминаниями Миллера –  0.010635076467578003


Судя по коэффициенту корреляции - цена закрытия не зависит от количества упоминаний Миллера

### Упоминаний какого газопровода в статьях больше:
 - "северный поток"
 - "турецкий поток"?

In [10]:
pattern1 = re.compile(r'(северный поток)', re.IGNORECASE)
pattern2 = re.compile(r'(турецкий поток)', re.IGNORECASE)

res1, res2 = 0, 0

for i in range(len(result['text'][:])):
    search_str = texts['text'][i]
    res = re.findall(pattern1, search_str)
    res1 += len(res)
    res = re.findall(pattern2, search_str)
    res2 += len(res)
    
print("Северный поток - ", res1)
print("Турецкий поток - ", res2)

Северный поток -  15
Турецкий поток -  39


Турецкий поток упоминается более чем в два раза чаще, чем северный поток

### Кого упоминают чаще:
 - Алексея Миллера
 - Владимира Путина?

In [11]:
pattern_miller = re.compile(r'(Миллер)', re.IGNORECASE)
pattern_putin = re.compile(r'(Путин)', re.IGNORECASE)

res_miller, res_putin = 0, 0

for i in range(len(result['text'][:])):
    search_str = texts['text'][i]
    res = re.findall(pattern_miller, search_str)
    res_miller += len(res)
    res = re.findall(pattern_putin, search_str)
    res_putin += len(res)
    
print("Упоминания Миллера - ", res_miller)
print("Упоминания Путина - ", res_putin)

Упоминания Миллера -  147
Упоминания Путина -  72


Миллер упоминается более чем в два раза чаще, чем Путин

### О каких санкциях пишут в статьях?

In [12]:
all_tokens = []
for i in tqdm(range(len(texts['text'][:]))):
    tokens = nltk.word_tokenize(texts['text'][i])
    text = nltk.Text(tokens)
    all_tokens.extend(text)
text = nltk.Text(all_tokens)   

text.concordance('санкция') 

100%|██████████| 1203/1203 [00:01<00:00, 1147.36it/s]


Displaying 25 of 67 matches:
нополия новый редакция американский санкция ставить угроза зарубежный нефтяной 
 ” участок интересоваться shell изз санкция иностранец 2014 год шельф россия по
 ” участок интересоваться shell изз санкция иностранец 2014 год шельф россия по
й компания рф наблюдаться число изз санкция эксперт полагать сила опыт организа
й компания рф наблюдаться число изз санкция эксперт полагать сила опыт организа
изингодатель почувствовать западный санкция “ ъ ” рассказать гендиректор « газп
рдить правомерность ввести совет ес санкция отношение « роснефти » брюссель зап
ение дело недовольный грозить банка санкция территория украина два крупный росс
ение дело недовольный грозить банка санкция территория украина казахстан подтве
южнокиринский месторождение попасть санкция по мнение эксперт пока речь идти по
ение направление серьёзно осложнять санкция сша украина подать иск киевский суд
ка повышение ликвидность находиться санкция банка если полторадва год убыток пр
нс 65 собст

## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
* составьте обучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [13]:
# добавляем новый столбец 0-1 понижение-повышение цены
new_column = [] 
for i in range(len(pr_all['ClosingPrice']) - 1): 
    new_column.append(0 if pr_all['ClosingPrice'][i] < pr_all['ClosingPrice'][i + 1] else 1) 
new_column.append(0) 

pr_all['ClosingPrice_bin'] = new_column

In [14]:
# соединяем с нашими текстами
df = pd.merge(result[['Date', 'text', 'text_original']], pr_all[['Date', 'ClosingPrice_bin']], on='Date')
df.head()

,Date,text,text_original,ClosingPrice_bin
0,09.11.2017,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0
1,08.11.2017,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1
2,01.11.2017,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1
3,30.10.2017,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1
4,23.10.2017,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1


In [15]:
date_format = "%d.%m.%Y"

for i in tqdm(range(len(df['Date']))):
    date = datetime.strptime(df['Date'][i], date_format)
    df['Date'][i] = date
df.rename(columns={'ClosingPrice_bin': 'target'}, inplace=True)
df.head()

100%|██████████| 1159/1159 [01:17<00:00, 14.94it/s]


,Date,text,text_original,target
0,2017-11-09 00:00:00,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0
1,2017-11-08 00:00:00,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1
2,2017-11-01 00:00:00,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1
3,2017-10-30 00:00:00,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1
4,2017-10-23 00:00:00,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1


In [16]:
# делим на трейн/тест
df_train, df_test = df[df['Date'] < datetime(2016, 1, 1)], df[df['Date'] >= datetime(2016, 1, 1)]
print('train size %d, test size %d' % (df_train.shape[0], df_test.shape[0]))

train size 901, test size 258


In [17]:
df_train.index = range(len(df_train.index))
df_test.index = range(len(df_test.index))

### word tf-idf + MultinomialNB

In [18]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(1,1), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

# fit our prediction model
model = MultinomialNB(alpha=0.5)
model.fit(X, y_train)
print('train f1_score', metrics.f1_score(y_train, model.predict(X)))
print('train accuracy', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test f1_score', metrics.f1_score(y_test, model.predict(X_test)))
print('test accuracy', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (901, 7736)
train f1_score 0.9785794813979707
train accuracy 0.978912319644839
test f1_score 0.415686274509804
test accuracy 0.42248062015503873


### word tf-idf + MultinomialNB + GridSearchCV alpha

In [19]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(2,4), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

grid = GridSearchCV(MultinomialNB(), # our model 
                    param_grid={'alpha': np.linspace(-1.0, 2.0, 31)})
 
grid.fit(X, y_train)
print('best params', grid.best_params_)
print('best estimator', grid.best_score_)
model = grid.best_estimator_


# fit our prediction model
# model = MultinomialNB(alpha=0.9)
model.fit(X, y_train)
print('train f1_score', metrics.f1_score(y_train, model.predict(X)))
print('train accuracy', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test f1_score', metrics.f1_score(y_test, model.predict(X_test)))
print('test accuracy', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (901, 167792)
best params {'alpha': 0.10000000000000009}
best estimator 0.5405105438401776
train f1_score 1.0
train accuracy 1.0
test f1_score 0.502092050209205
test accuracy 0.5387596899224806


### char tf-idf  + LogisticRegression + GridSearchCV (C, penalty)

In [20]:
SEED = 2018

# build feature matrix from 3-grams
# with l2-normalization and smoothed idf
vec = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(3,3), norm='l2', use_idf=True, smooth_idf=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# Logistic Regression classifier has several hyperparams
grid = GridSearchCV(LogisticRegression(random_state=SEED), # our model 
                   param_grid={'C': np.logspace(0,5,10),
                               'penalty': ['l1', 'l2']}, 
                    scoring='f1', # our perfomance measure 
                    n_jobs=-1, # multithread 
                    cv=5, # 5-fold stratified cross-validation 
                    verbose=True, return_train_score=True)

grid.fit(X, y_train)
print('best params', grid.best_params_)
print('best estimator', grid.best_score_)
model = grid.best_estimator_

# grid.best_estimator_ is already fitted on whole train dataset
print('train', metrics.f1_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
print('test', metrics.f1_score(y_test, model.predict(X_test)))

feature matrix shape (901, 10806)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.7s finished


best params {'C': 12.91549665014884, 'penalty': 'l1'}
best estimator 0.5194099456597111
train 0.9977678571428571
test 0.49420849420849416


### char tf-idf  + exclude 3-grams appeared in less then 10 texts + LogisticRegression + GridSearchCV (C, penalty)

In [21]:
# baseline 3

# build feature matrix from 3-grams
vec = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(3,3), 
                      min_df=10, # exclude 3-grams appeared in less then 10 texts. 
                      use_idf=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# optimize  hyperparams
grid = GridSearchCV(LogisticRegression(random_state=SEED), 
                   param_grid={'C': np.logspace(0,5,10), 
                               'penalty': ['l1', 'l2']}, 
                    scoring='f1', n_jobs=-1, cv=5, verbose=True, return_train_score=True)

grid.fit(X, y_train)
print(grid.best_params_)
print(grid.best_score_)
model = grid.best_estimator_
print('train', metrics.f1_score(y_train, model.predict(X)))

X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
print('test', metrics.f1_score(y_test, model.predict(X_test)))

feature matrix shape (901, 3848)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.7s finished


{'C': 3.5938136638046276, 'penalty': 'l1'}
0.5205123462060199
train 0.824188129899216
test 0.46268656716417916


## accuracy = 0.5387 получилось максимум на word tf-idf + MultinomialNB + GridSearchCV alpha

## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

In [22]:
df.head()

,Date,text,text_original,target
0,2017-11-09 00:00:00,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0
1,2017-11-08 00:00:00,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1
2,2017-11-01 00:00:00,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1
3,2017-10-30 00:00:00,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1
4,2017-10-23 00:00:00,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1


### Оставим только существительные

In [23]:
morph = pymorphy2.MorphAnalyzer()

df['text_nouns'] = df['text']
for i in tqdm(range(len(df['text']))):
    sent = df['text'][i]
    res = []
    for word in sent.split():
        if morph.parse(word)[0].tag.POS == 'NOUN':
            res.append(word)
    df['text_nouns'][i] = ' '.join(res)

100%|██████████| 1159/1159 [01:59<00:00,  9.71it/s]


In [24]:
df.head()

,Date,text,text_original,target,text_nouns
0,2017-11-09 00:00:00,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0,компания решение украина конец декабрь решение...
1,2017-11-08 00:00:00,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1,жалоба партнёр налогообложение проект рф сниже...
2,2017-11-01 00:00:00,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1,редакция санкция угроза проект компания гражда...
3,2017-10-30 00:00:00,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1,рынок рискменеджер itспециалист вадим кулик ра...
4,2017-10-23 00:00:00,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1,новатэк год камчатка терминал проблема обеспеч...


In [25]:
df_train, df_test = df[df['Date'] < datetime(2016, 1, 1)], df[df['Date'] >= datetime(2016, 1, 1)]
print('train size %d, test size %d' % (df_train.shape[0], df_test.shape[0]))

train size 901, test size 258


In [26]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(2,4), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text_nouns)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

grid = GridSearchCV(MultinomialNB(), # our model 
                    param_grid={'alpha': np.linspace(-1.0, 2.0, 31)})
 
grid.fit(X, y_train)
print('best params', grid.best_params_)
print('best estimator', grid.best_score_)
model = grid.best_estimator_


# fit our prediction model
model.fit(X, y_train)
print('train f1_score', metrics.f1_score(y_train, model.predict(X)))
print('train accuracy', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text_nouns)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test f1_score', metrics.f1_score(y_test, model.predict(X_test)))
print('test accuracy', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (901, 89180)
best params {'alpha': 0.10000000000000009}
best estimator 0.5194228634850167
train f1_score 1.0
train accuracy 1.0
test f1_score 0.4645669291338583
test accuracy 0.4728682170542636


### Оставим только существительные и глаголы

In [27]:
df['text_nouns_verbs'] = df['text']
for i in tqdm(range(len(df['text']))):
    sent = df['text'][i]
    res = []
    for word in sent.split():
        if (morph.parse(word)[0].tag.POS == 'NOUN') or (morph.parse(word)[0].tag.POS == 'VERB'):
            res.append(word)
    df['text_nouns_verbs'][i] = ' '.join(res)

100%|██████████| 1159/1159 [02:07<00:00,  9.11it/s]


In [28]:
df.head()

,Date,text,text_original,target,text_nouns,text_nouns_verbs
0,2017-11-09 00:00:00,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0,компания решение украина конец декабрь решение...,компания решение украина конец декабрь решение...
1,2017-11-08 00:00:00,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1,жалоба партнёр налогообложение проект рф сниже...,жалоба партнёр налогообложение проект рф сниже...
2,2017-11-01 00:00:00,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1,редакция санкция угроза проект компания гражда...,редакция санкция угроза проект компания гражда...
3,2017-10-30 00:00:00,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1,рынок рискменеджер itспециалист вадим кулик ра...,рынок рискменеджер itспециалист вадим кулик ра...
4,2017-10-23 00:00:00,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1,новатэк год камчатка терминал проблема обеспеч...,новатэк год камчатка терминал проблема обеспеч...


In [29]:
df_train, df_test = df[df['Date'] < datetime(2016, 1, 1)], df[df['Date'] >= datetime(2016, 1, 1)]
print('train size %d, test size %d' % (df_train.shape[0], df_test.shape[0]))

train size 901, test size 258


In [30]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(2,4), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text_nouns_verbs)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

grid = GridSearchCV(MultinomialNB(), # our model 
                    param_grid={'alpha': np.linspace(-1.0, 2.0, 31)})
 
grid.fit(X, y_train)
print('best params', grid.best_params_)
print('best estimator', grid.best_score_)
model = grid.best_estimator_

# fit our prediction model
model.fit(X, y_train)
print('train f1_score', metrics.f1_score(y_train, model.predict(X)))
print('train accuracy', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text_nouns_verbs)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test f1_score', metrics.f1_score(y_test, model.predict(X_test)))
print('test accuracy', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (901, 89711)
best params {'alpha': 0.10000000000000009}
best estimator 0.5049944506104328
train f1_score 1.0
train accuracy 1.0
test f1_score 0.45849802371541504
test accuracy 0.4689922480620155


### Оставим только те слова, которые входят в топики LDA

In [31]:
all_text = []
for i in tqdm(range(len(df['text'][:]))):
    sent = df['text'][i]
    all_text.append(sent)

100%|██████████| 1159/1159 [00:00<00:00, 37069.05it/s]


In [32]:
cv = CountVectorizer(max_features = 1000)
transformed_dataset = cv.fit_transform(all_text)
transformed_dataset

<1159x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 47698 stored elements in Compressed Sparse Row format>

In [33]:
tf_feature_names = cv.get_feature_names()
assert len(tf_feature_names) == 1000
tf_feature_names

['10',
 '100',
 '1000',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '20',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2020',
 '22',
 '24',
 '25',
 '27',
 '30',
 '40',
 '400',
 '49',
 '50',
 '500',
 '70',
 '75',
 'ag',
 'cnpc',
 'dujos',
 'energy',
 'eni',
 'eon',
 'gas',
 'gazprom',
 'germania',
 'lietuvos',
 'nis',
 'nord',
 'opal',
 'reuters',
 'shell',
 'south',
 'statoil',
 'stream',
 'total',
 'wintershall',
 'август',
 'автономный',
 'агентство',
 'актив',
 'активно',
 'активный',
 'акционер',
 'акция',
 'александр',
 'алексей',
 'альянс',
 'американский',
 'аналитик',
 'аналитика',
 'аналогичный',
 'андрей',
 'антимонопольный',
 'апрель',
 'арбитраж',
 'арбитражный',
 'аукцион',
 'аэропорт',
 'база',
 'балтийский',
 'банк',
 'банка',
 'банковский',
 'белоруссия',
 'белтрансгаз',
 'бензин',
 'бизнес',
 'бизнесмен',
 'биржа',
 'ближний',
 'блок',
 'болгария',
 'большой',
 'борьба',
 'будущее',
 'бумага',
 'бывший',
 'быт

In [34]:
lda = lda(n_topics=20, max_iter=50, learning_method='batch', random_state=1337)
lda.fit(transformed_dataset)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=1337,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [35]:
def print_top_words(model, feature_names, n_top_words):
    res = ''
    for topic_idx, topic in enumerate(model.components_):
        res += ' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        res += ' '
    return res

In [36]:
n_top_words = 20
lda
res = print_top_words(lda, tf_feature_names, n_top_words)
themes = ' '.join(list(set(res.split())))
print(' '.join(list(set(res.split()))))
print(len(' '.join(list(set(res.split()))).split()))

дивиденд вопрос производитель фонд производство украина создание составить владимир тнквр миллион прибыль как бизнесмен медиа отношение новатэк льгота идея газпроммедиа история вэб газпрома литва поставка рао совет договор компания пхг геннадий путин свой директор развитие подписать интерфакс завод газа гендиректор чистый акционер nord stream сообщать бензин совместный гэх монополия доля комитет федеральный банк экспорт бывший тендер нпз интер вырасти пакет lietuvos закупка национальный снижение новый спг александр общий антимонопольный европейский принять годовой метр правительство втб немецкий также построить 2013 проект участок контрольный грэс добыча рынок лукойл станция капитал миллер eni холдинг другой налог предлагать south сообщение генеральный москва два поток иск участие gazprom заявить итог мощность крупный налоговый московский источник алексей россия труба рейтинг сторона продажа он строительство привлечь заявка готовый топливо евро ссылка год на подземный тот газовый прода

In [37]:
df['text_lda'] = df['text']
for i in tqdm(range(len(df['text']))):
    sent = df['text'][i]
    res = []
    for word in sent.split():
        if word in themes:
            res.append(word)
    df['text_lda'][i] = ' '.join(res)

100%|██████████| 1159/1159 [12:00:25<00:00, 37.30s/it]


In [38]:
df.head()

,Date,text,text_original,target,text_nouns,text_nouns_verbs,text_lda
0,2017-11-09 00:00:00,компания рассчитывать решение газовый спор укр...,Компания рассчитывает на решение по газовому с...,0,компания решение украина конец декабрь решение...,компания решение украина конец декабрь решение...,компания решение газовый спор украина решение ...
1,2017-11-08 00:00:00,как предполагать “ъ” «газпром» воспользоваться...,"Как и предполагал “Ъ”, «Газпром», воспользова...",1,жалоба партнёр налогообложение проект рф сниже...,жалоба партнёр налогообложение проект рф сниже...,как проект рф снижение налог добыча ндпить сво...
2,2017-11-01 00:00:00,новый редакция американский санкция ставить уг...,Новая редакция американских санкций ставит по...,1,редакция санкция угроза проект компания гражда...,редакция санкция угроза проект компания гражда...,новый ставить нефтяной проект российский компа...
3,2017-10-30 00:00:00,как стать известно “ъ” известный рынок рискмен...,"Как стало известно “Ъ”, известный на рынке ри...",1,рынок рискменеджер itспециалист вадим кулик ра...,рынок рискменеджер itспециалист вадим кулик ра...,как стать рынок газпромбанк о правление год вт...
4,2017-10-23 00:00:00,новатэк который пять год собираться запустить ...,"НОВАТЭК, который через пять лет собирается за...",1,новатэк год камчатка терминал проблема обеспеч...,новатэк год камчатка терминал проблема обеспеч...,новатэк который год крупный спг готовый газ по...


In [39]:
df_train, df_test = df[df['Date'] < datetime(2016, 1, 1)], df[df['Date'] >= datetime(2016, 1, 1)]
print('train size %d, test size %d' % (df_train.shape[0], df_test.shape[0]))

train size 901, test size 258


In [40]:
# build binary feature matrix from BoW model
vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(2,4), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text_lda)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.target)
y_train = label_enc.transform(df_train.target)

grid = GridSearchCV(MultinomialNB(), # our model 
                    param_grid={'alpha': np.linspace(-1.0, 2.0, 31)})
 
grid.fit(X, y_train)
print('best params', grid.best_params_)
print('best estimator', grid.best_score_)
model = grid.best_estimator_


# fit our prediction model

model.fit(X, y_train)
print('train f1_score', metrics.f1_score(y_train, model.predict(X)))
print('train accuracy', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text_lda)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.target)
print('test f1_score', metrics.f1_score(y_test, model.predict(X_test)))
print('test accuracy', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (901, 56930)
best params {'alpha': 0.10000000000000009}
best estimator 0.5427302996670367
train f1_score 1.0
train accuracy 1.0
test f1_score 0.465648854961832
test accuracy 0.4573643410852713


### Качество улучшить не удалось, нам кажется из-за того, что очень маленький датасет (да и было оно не намного лучше рандома скажем честно)

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

